In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder

In [2]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [3]:
soup = BeautifulSoup(source, 'xml')


In [4]:
table=soup.find('table')

In [5]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']

In [6]:
pcode=[]
borough=[]
nhood=[]

In [7]:
for tr_cell in table.find_all('tr'):
    for td_cell in tr_cell.find_all('td'):
        for b in td_cell.find_all('b'):
            pcode.append(b.text.strip())
            
            brh=str(str(td_cell).split('<br/>')[1]).split('\n')[0]
            soup=BeautifulSoup(brh,'lxml')
            borough.append(soup.text)
            
            if soup.text=='Not assigned':
                nhood.append("Not assigned")            
            else:
                nh=str(str(str(str(td_cell).split('<br/>')[2:]).split('\\n')[0]).split('(')[1]).split(')')[0]
                if "['" in nh:
                    nh=str(str(str(str(str(td_cell).split('<br/>')[2:]).split('\\n')[0]).split("['")[1]).split('(')[1]).split(')')[0]
                soup=BeautifulSoup(nh,'lxml')
                nhood.append(soup.text)
# print(pcode)
# print(borough)
# print(nhood)

In [8]:
df = pd.DataFrame(list(zip(pcode,borough,nhood)),columns = column_names)
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [9]:
df1=df[df['Borough']!='Not assigned']
df1.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [10]:
dfgeo = pd.read_csv("https://cocl.us/Geospatial_data")
dfgeo.rename(columns={'Postal Code': 'Postalcode'}, inplace=True)

In [11]:
df2 = pd.merge(df, dfgeo, on="Postalcode", how='left')

In [12]:
df2.loc[df2['Postalcode'] == 'M5G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude
40,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [13]:
df2.loc[df2['Postalcode'] == 'M9V']

,Postalcode,Borough,Neighborhood,Latitude,Longitude
143,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437


In [14]:
df2 = pd.merge(df1,dfgeo,on='Postalcode')
df2.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [15]:
from geopy.geocoders import Nominatim
import folium

In [16]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Postalcode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.
